In [1]:
# load .env file
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def mystery(x: int, y: int) -> int: 
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

In [4]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o-mini")
response = llm.predict_and_call(
    [add_tool, mystery_tool], 
    "Tell me the output of the mystery function on 2 and 9", 
    verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121


In [5]:
from llama_index.core import SimpleDirectoryReader
# load documents
documents = SimpleDirectoryReader(input_files=["The Era of Experience Paper.pdf"]).load_data()

In [6]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [7]:
print(nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: The Era of Experience Paper.pdf
file_path: The Era of Experience Paper.pdf
file_type: application/pdf
file_size: 226196
creation_date: 2025-04-21
last_modified_date: 2025-04-21

Welcome to the Era of Experience
David Silver, Richard S. Sutton*
Abstract
We stand on the threshold of a new era in artificial intelligence that promises to achieve an unprece-
dented level of ability. A new generation of agents will acquire superhuman capabilities by learning pre-
dominantly from experience. This note explores the key characteristics that will define this upcoming era.
The Era of Human Data
Artificial intelligence (AI) has made remarkable strides over recent years by training on massive amounts of
human-generated data and fine-tuning with expert human examples and preferences. This approach is exem-
plified by large language models (LLMs) that have achieved a sweeping level of generality. A single LLM
can now perform tasks spanning from writing poetry and solving phys

In [8]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [9]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"}
        ]
    )
)

response = query_engine.query(
    "Who is the author of the paper?", 
)

In [10]:
print(str(response))

The author of the paper is the AI community.


In [11]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'The Era of Experience Paper.pdf', 'file_path': 'The Era of Experience Paper.pdf', 'file_type': 'application/pdf', 'file_size': 226196, 'creation_date': '2025-04-21', 'last_modified_date': '2025-04-21'}


In [12]:
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_query(
    query: str, 
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.
    
    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.
    
    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response
    

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

In [13]:
llm = OpenAI(model="gpt-4o-mini", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool], 
    "What is the main idea described in paper 2?", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "main idea", "page_numbers": ["2"]}
=== Function Output ===
The main idea is about the potential of experiential learning in AI systems, emphasizing the benefits of agents learning from continuous streams of experience rather than isolated interactions. It discusses how agents can adapt and improve over time by carrying information across their entire experience, enabling them to plan for future goals and achieve long-term outcomes. The focus is on harnessing the power of experiential data to break through the limitations of current human-centric AI systems and advance towards creating truly superhuman agents.


In [14]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'The Era of Experience Paper.pdf', 'file_path': 'The Era of Experience Paper.pdf', 'file_type': 'application/pdf', 'file_size': 226196, 'creation_date': '2025-04-21', 'last_modified_date': '2025-04-21'}


In [22]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of a particular document. "
    ),
)

In [16]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "Actions vs Observations described in page 3?", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "Actions vs Observations", "page_numbers": ["3"]}
=== Function Output ===
Agents in the era of experience will act autonomously in the real world, interacting with their environment through various actions such as using user interfaces, executing code, calling APIs, monitoring sensors, operating equipment remotely, and conducting experiments. These actions enable the agents to explore, adapt to changing environments, and discover strategies independently. In contrast, observations in this era involve the agent analyzing real-world data, running simulations, suggesting experiments, and interventions to maximize long-term success towards a specified goal.


In [17]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '3', 'file_name': 'The Era of Experience Paper.pdf', 'file_path': 'The Era of Experience Paper.pdf', 'file_type': 'application/pdf', 'file_size': 226196, 'creation_date': '2025-04-21', 'last_modified_date': '2025-04-21'}


In [23]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "What is the summary of the document?", 
    verbose=True
)

ValueError: Expected at least one tool call, but got 0 tool calls.